In [ ]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('../')

In [ ]:
from src.ADB import ADB 
from src.Webcam import Webcam 
from src.Audio import Audio 
from src.StreamManager import StreamManager 
from PIL import ImageTk as PIL_ImageTk
import tkinter as tk
import tkinter.messagebox
from datetime import datetime
import time
import threading

In [ ]:
def init_stream_manager():
    sm=StreamManager()
    sm.register_stream(stream_class=ADB,stream_type='video',stream_name='ADB')
    sm.register_stream(stream_class=Webcam,stream_type='video',device_id=0,stream_name='Webcam-1')
    sm.register_stream(stream_class=Audio,stream_type='audio',stream_name='Microphone',bitrate=44100,frames_per_buffer=1024,channels=2)
    sm.set_video_record_size('Webcam-1',(480,320))
    sm.set_video_record_size('ADB',(295,640))
    sm.start()
    return sm

In [ ]:
class MainWindow():
    def toggle_recording(self):
        if self.is_recording:
            self.is_recording=False
            self.startstop_button['text']="Start recording"
            self.startstop_button.config(state="disabled")
            self.sm.stop_recording()
            self.startstop_button.config(state="normal")
        else:
            self.is_recording=True
            session_name=datetime.now().strftime('%Y%m%d_%H%M%S')
            self.startstop_button['text']=f"Stop recording {session_name}"
            self.startstop_button.config(state="disabled")
            self.sm.start_recording(session_name)
            self.startstop_button.config(state="normal")
    def on_close(self):
        self.is_recording=False
        self.is_running=False
        self.sm.stop()
        self.root.destroy()
    def create_main_window(self):
        root = tk.Tk() # create root window
        root.title("Three in one") # title of the GUI window
        root.maxsize(1200, 900) # specify the max size the window can expand to
        root.config(bg="black") # specify background color
        root.wm_protocol("WM_DELETE_WINDOW", self.on_close)
        return root
    def create_frame(self,width,height,row,column,bg_color):
        frame = tk.Frame(self.root, width=width, height=height, bg=bg_color)
        frame.grid(row=row, column=column, padx=0, pady=0)
        return frame
    
    def create_left_frame(self,width,height):
        frame=self.create_frame(width=width, height=height, row=0, column=0, bg_color='blue')
        self.startstop_button=tk.Button(frame, text ="Start record",command=self.toggle_recording)
        self.startstop_button.config(height=5,width=20)
        self.startstop_button.pack()
        return frame
    
    def create_mid_frame(self,width,height):
        frame=self.create_frame(width=width, height=height, row=0, column=1, bg_color='green')
        frame.canvas=tk.Canvas(frame, height=height, width=width)
        frame.canvas.img=None
        frame.canvas.image_id = frame.canvas.create_image(0, 0, anchor='nw',image=frame.canvas.img)
        frame.canvas.pack()
        frame.canvas.size=(width,height)
        return frame
        
    def create_right_frame(self,width,height):
        #THINK ABOUT RIGHT FRAME TOP BOTTOm
        
        frame=self.create_frame(width=width, height=height, row=0, column=2, bg_color='red')
        frame.top_canvas   =tk.Canvas(frame, height=int(height/2), width=width)
        frame.top_canvas.img=None
        frame.top_canvas.image_id = frame.top_canvas.create_image(0, 0, anchor='nw',
                                                                  image=frame.top_canvas.img)
        frame.top_canvas.pack()
        frame.top_canvas.width=width
        frame.top_canvas.size=(width,int(height/2))
        
        frame.bottom_canvas=tk.Canvas(frame, height=int(height/2), width=width)
        frame.bottom_canvas.img=None
        frame.bottom_canvas.image_id = frame.bottom_canvas.create_image(0, 0, anchor='nw',
                                                                        image=frame.bottom_canvas.img)
        frame.bottom_canvas.pack()     
        frame.bottom_canvas.size=(width,int(height/2))
        return frame
    
    def update_mid_frame(self):
        frame=self.sm.get_current_frame(stream='ADB',frame_size=self.mid_frame.canvas.size)
        if frame is not None:
            self.mid_frame.canvas.img = PIL_ImageTk.PhotoImage(frame)
            self.mid_frame.canvas.itemconfigure(self.mid_frame.canvas.image_id, 
                                                image=self.mid_frame.canvas.img)
    
    def update_right_frame(self):
        frame=self.sm.get_current_frame(stream='Webcam-1',frame_size=self.right_frame.top_canvas.size)
        if frame is not None:
            self.right_frame.top_canvas.img = PIL_ImageTk.PhotoImage(frame)
            self.right_frame.top_canvas.itemconfigure(self.right_frame.top_canvas.image_id, 
                                                      image=self.right_frame.top_canvas.img)

        frame=self.sm.get_current_frame(stream='Webcam-1',frame_size=self.right_frame.bottom_canvas.size)
        if frame is not None:
            self.right_frame.bottom_canvas.img = PIL_ImageTk.PhotoImage(frame)
            self.right_frame.bottom_canvas.itemconfigure(self.right_frame.bottom_canvas.image_id, 
                                                         image=self.right_frame.bottom_canvas.img)
    
    def update_frames(self):
        if not self.is_running:
            return
        
        try:
            self.update_mid_frame()
            self.update_right_frame()
        except Exception as e:  # missing or corrupt image
            print(e)
            pass

        self.root.after(1, self.update_frames) 
       

    
    def __init__(self,sm):
        self.is_running=True
        self.is_recording=False
        self.sm=sm
        self.root=self.create_main_window()
        self.left_frame =self.create_left_frame(width=240, height=640)
        self.mid_frame  =self.create_mid_frame(width=295, height=640)        
        self.right_frame=self.create_right_frame(width=480, height=640)
        self.update_frames()
        self.root.mainloop()
        


In [ ]:
sm=init_stream_manager()

In [ ]:
main_window=MainWindow(sm)